In [5]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/Users/ajakobs/Documents/python_packages/ecospold2matrix/')
sys.path.append('/Users/ajakobs/Documents/python_packages/pylcaio/src/')
import ecospold2matrix as e2m
import pymrio
import pylcaio

import os
import gzip
import pickle
import pandas as pd
import itertools

# from importlib import reload
# import pdb
import scipy.sparse

In [6]:
%%time

#run ecospold2matrix
ecospold_dir = '/Users/ajakobs/Documents/data/ecoinvent/ecoinvent 3.9_cutoff_ecoSpold02/'
project_name = 'ecoinvent_39'
out_dir = '/Users/ajakobs/Documents/data/ecospold2matrix_matrices/'

parser = e2m.Ecospold2Matrix(ecospold_dir, project_name, out_dir, positive_waste=True, nan2null=True,
                            characterisation_file=
        '/Users/ajakobs/Documents/data/ecoinvent/ecoinvent 3.9_LCIA_implementation/LCIA Implementation 3.9.xlsx', )
parser.ecospold_to_Leontief(fileformats='Pandas',with_absolute_flows=False)

2023-02-27 22:17:58,467 - ecoinvent_39 - INFO - Ecospold2Matrix Processing
fatal: not a git repository (or any of the parent directories): .git
2023-02-27 22:17:58,527 - ecoinvent_39 - INFO - Project name: ecoinvent_39
2023-02-27 22:17:58,529 - ecoinvent_39 - INFO - Unit process and Master data directory: /Users/ajakobs/Documents/data/ecoinvent/ecoinvent 3.9_cutoff_ecoSpold02/
2023-02-27 22:17:58,530 - ecoinvent_39 - INFO - Data saved in: /Users/ajakobs/Documents/data/ecospold2matrix_matrices
2023-02-27 22:17:58,532 - ecoinvent_39 - INFO - Sign conventions changed to make waste flows positive
2023-02-27 22:17:58,534 - ecoinvent_39 - INFO - Replace Not-a-Number instances with 0.0 in all matrices
2023-02-27 22:17:58,535 - ecoinvent_39 - INFO - Pickle intermediate results to files
2023-02-27 22:17:58,537 - ecoinvent_39 - INFO - Order processes based on: ISIC, activityName
2023-02-27 22:17:58,538 - ecoinvent_39 - INFO - Order elementary exchanges based on: comp, name, subcomp
rm: ecoinvent

CPU times: user 10min 11s, sys: 47.2 s, total: 10min 58s
Wall time: 8h 44min 15s


In [7]:
%%time
path_ecoinvent = '/Users/ajakobs/Documents/data/ecospold2matrix_matrices/ecoinvent_39Pandas_symmNorm.gz.pickle'
with gzip.open(path_ecoinvent,'rb') as f:
    ecoinvent = pd.read_pickle(f)


CPU times: user 6.64 s, sys: 1.52 s, total: 8.16 s
Wall time: 8.67 s


In [8]:
%%time
EXIOBASE_path = '/Users/ajakobs/Documents/data/EXIOBASE/EX_3_8_1/IOT_2015_pxp/'
io = pymrio.parse_exiobase3(EXIOBASE_path)

CPU times: user 1min 6s, sys: 6.76 s, total: 1min 12s
Wall time: 1min 13s


In [10]:
%%time
database_loader = pylcaio.DatabaseLoader(lca_database_processed=ecoinvent,
                                         io_database_processed=io, 
                                         lca_database_name_and_version='ecoinvent3.9',
                                         io_database_name_and_version='exiobase3')


CPU times: user 9.08 s, sys: 1.38 s, total: 10.5 s
Wall time: 10.9 s


In [11]:
%%time
lcaio_object = database_loader.combine_ecoinvent_exiobase(complete_extensions=False, 
                                        impact_world=False, regionalized=False)

No path for the capital folder was provided. Capitals will not be endogenized
CPU times: user 11min 39s, sys: 21.1 s, total: 12min
Wall time: 1min 52s


In [13]:
%%time
lcaio_object.hybridize('STAM', capitals=False)

Empty!


/Users/ajakobs/Documents/python_packages/pylcaio/src/pylcaio.py:766: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.H = self.H.append([self.H] * (self.number_of_countries_IO + self.number_of_RoW_IO - 1))
/Users/ajakobs/Documents/python_packages/pylcaio/src/pylcaio.py:932: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.G = self.G.append([self.G] * (self.number_of_countries_IO + self.number_of_RoW_IO - 1))


CPU times: user 1h 59min 43s, sys: 5min 55s, total: 2h 5min 38s
Wall time: 21min 24s


In [14]:
%%time
lcaio_object.save_system('pickle')

CPU times: user 16.1 s, sys: 1.03 s, total: 17.2 s
Wall time: 17.9 s


In [18]:
lcaio_object.A_io_f

<9800x21255 sparse matrix of type '<class 'numpy.float64'>'
	with 596408 stored elements in Compressed Sparse Row format>

In [19]:
%%time
with gzip.open('/Users/ajakobs/Documents/python_packages/pylcaio/src/Databases/ecoinvent3.9_exiobase3/lcaio_object.pickle.zip', 'wb') as fh:
    pickle.dump(lcaio_object, fh)

CPU times: user 2min 45s, sys: 11.1 s, total: 2min 56s
Wall time: 3min 3s


In [ ]:
del lcaio_object